In [1]:
import pandas as pd
import numpy as np
import os
import requests
import env
import wrangle as w
import explore as exp

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

In [3]:
# Example usage:
app_token = env.app_token
year_to_retrieve = '2022'
max_req = 2000  # Specify the maximum number of observations to retrieve


In [4]:
# get_data function retrieves and caches the data in csv format ("nyc_collisions_{year}")
# subsequent runs will load the csv as to be respectful of the resources provided for free.
df = w.get_data(year_to_retrieve, app_token)
df.head()
#for year 2022, retrieval took about 00:1:10 (mins) and 103,875 records

CSV file for 2022 already exists. Loading data from the CSV.


,crash_date,crash_time,latitude,longitude,location,on_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,collision_id,vehicle_type_code1,off_street_name,vehicle_type_code2,borough,zip_code,cross_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code_4,vehicle_type_code_5
0,2022-01-01T00:00:00.000,7:33,40.771477,-73.91824,"{'latitude': '40.771477', 'longitude': '-73.91...",HOYT AVENUE NORTH,0,0,0,0,0,0,0,0,Unspecified,Unspecified,4491400,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01T00:00:00.000,4:30,NaN,NaN,NaN,Southern parkway,0,0,0,0,0,0,0,0,Pavement Slippery,Unspecified,4491626,Sedan,Jfk expressway,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01T00:00:00.000,7:57,NaN,NaN,NaN,WESTCHESTER AVENUE,0,0,0,0,0,0,0,0,Unspecified,NaN,4491734,Sedan,SHERIDAN EXPRESSWAY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01T00:00:00.000,5:17,40.746930,-73.84866,"{'latitude': '40.74693', 'longitude': '-73.848...",GRAND CENTRAL PKWY,1,0,0,0,0,0,1,0,Passing or Lane Usage Improper,Unsafe Lane Changing,4491857,Sedan,NaN,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01T00:00:00.000,1:30,40.819157,-73.96038,"{'latitude': '40.819157', 'longitude': '-73.96...",HENRY HUDSON PARKWAY,0,0,0,0,0,0,0,0,Unspecified,NaN,4491344,Sedan,NaN,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Convert 'crash_date' and 'crash_time' to datetime and combine them into a single column
df['crash_datetime'] = pd.to_datetime(df['crash_date'].str[:10] + ' ' + df['crash_time'], format='%Y-%m-%d %H:%M')
df['crash_date'] = pd.to_datetime(df['crash_date'])


In [6]:
df = w.initial_reorder_cols(df)

In [7]:
# Create a mask where we filter out any observation with a value in vehicle 3 +
# This will leave us with only the observations that have 2 vehicles involved
condition = (df['vehicle_type_code_3'].isnull() &
             df['vehicle_type_code_4'].isnull() &
             df['vehicle_type_code_5'].isnull() &
             df['contributing_factor_vehicle_3'].isnull()
             )

# Apply the condition to filter the DataFrame
df = df[condition]

# Reset the index if needed
df.reset_index(drop=True, inplace=True)

df.drop(columns=['vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5', 'contributing_factor_vehicle_3', 'contributing_factor_vehicle_4', 'contributing_factor_vehicle_5' ], inplace=True)


In [8]:
# filter only keep observations with a value in vehicle_type_code1 and 2
# this will leave us with collisions of 2 vehicles only. 
condition = (df['vehicle_type_code1'].notnull() &
             df['vehicle_type_code2'].notnull())
df = df[condition]

# Reset the index if needed
df.reset_index(drop=True, inplace=True)

In [9]:
df.columns

Index(['crash_datetime', 'crash_date', 'crash_time', 'collision_id',
       'latitude', 'longitude', 'location', 'on_street_name',
       'cross_street_name', 'off_street_name', 'borough', 'zip_code',
       'number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'vehicle_type_code1', 'contributing_factor_vehicle_1',
       'vehicle_type_code2', 'contributing_factor_vehicle_2'],
      dtype='object')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58537 entries, 0 to 58536
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_datetime                 58537 non-null  datetime64[ns]
 1   crash_date                     58537 non-null  datetime64[ns]
 2   crash_time                     58537 non-null  object        
 3   collision_id                   58537 non-null  int64         
 4   latitude                       53287 non-null  float64       
 5   longitude                      53287 non-null  float64       
 6   location                       53287 non-null  object        
 7   on_street_name                 45037 non-null  object        
 8   cross_street_name              13498 non-null  object        
 9   off_street_name                29066 non-null  object        
 10  borough                        37671 non-null  object        
 11  zip_code       

In [11]:
len(df)

58537

In [12]:
# Filter the DataFrame to exclude collisions involving pedestrians or cyclists
df = df[(df['number_of_pedestrians_injured'] == 0) & (df['number_of_pedestrians_killed'] == 0) & (df['number_of_cyclist_injured'] == 0) & (df['number_of_cyclist_killed'] == 0)]

In [15]:
df.number_of_persons_injured.unique()

array([ 0,  1,  2,  3,  4,  5,  6, 15,  7, 13,  8, 14,  9, 10, 11])

In [13]:
len(df)

54214

In [14]:
df = df.drop(columns=['cross_street_name', 'off_street_name', 'number_of_pedestrians_injured', 'number_of_pedestrians_killed', 'number_of_cyclist_injured', 'number_of_cyclist_killed'])

In [13]:
# Create a boolean mask for rows where injuries do not match
injury_mask = df['number_of_persons_injured'] != df['number_of_motorist_injured']

# Create a boolean mask for rows where there are no injuries or deaths involving pedestrians or cyclists
no_pedestrian_cyclist_injuries_mask = (
    (df['number_of_pedestrians_injured'] == 0) &
    (df['number_of_pedestrians_killed'] == 0) &
    (df['number_of_cyclist_injured'] == 0) &
    (df['number_of_cyclist_killed'] == 0)
)

# Combine the two masks using the & operator to get the final filtered DataFrame
filtered_df = df[injury_mask & no_pedestrian_cyclist_injuries_mask]

filtered_df

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
92,2022-01-01 17:07:00,2022-01-01,17:07,4491637,40.767815,-73.989510,"{'latitude': '40.767815', 'longitude': '-73.98...",WEST 55 STREET,NaN,10 AVENUE,MANHATTAN,10019.0,1,0,0,0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Unspecified,E-Scooter,Unspecified
151,2022-01-02 01:22:00,2022-01-02,1:22,4491810,40.590984,-73.975690,"{'latitude': '40.590984', 'longitude': '-73.97...",NaN,2911 86 STREET,NaN,BROOKLYN,11223.0,1,0,0,0,0,0,0,0,Sedan,Failure to Yield Right-of-Way,E-Scooter,Unspecified
175,2022-01-02 16:30:00,2022-01-02,16:30,4491667,NaN,NaN,NaN,NaN,61 EAST DRIVE,NaN,NaN,NaN,1,0,0,0,0,0,0,0,Carriage,Driver Inexperience,E-Scooter,Unspecified
270,2022-01-03 07:27:00,2022-01-03,7:27,4492150,40.871002,-73.893845,"{'latitude': '40.871002', 'longitude': '-73.89...",JEROME AVENUE,NaN,MORRIS AVENUE,BRONX,10468.0,1,0,0,0,0,0,0,0,E-Scooter,Driver Inattention/Distraction,Sedan,Unspecified
412,2022-01-04 12:45:00,2022-01-04,12:45,4492492,NaN,NaN,NaN,Edward L Grant Highwa,NaN,Plimpton Avenue,NaN,NaN,1,0,0,0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,E-Bike,Driver Inattention/Distraction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58347,2022-12-30 00:14:00,2022-12-30,0:14,4593919,40.645336,-73.986350,"{'latitude': '40.645336', 'longitude': '-73.98...",NaN,3483 FORT HAMILTON PARKWAY,NaN,BROOKLYN,11218.0,1,0,0,0,0,0,0,0,Taxi,Unspecified,E-Bike,Unspecified
58401,2022-12-30 16:10:00,2022-12-30,16:10,4595292,40.711970,-73.997780,"{'latitude': '40.71197', 'longitude': '-73.997...",MADISON STREET,NaN,OLIVER STREET,MANHATTAN,10038.0,1,0,0,0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,E-Scooter,Unspecified
58424,2022-12-30 15:30:00,2022-12-30,15:30,4597183,40.724873,-73.997240,"{'latitude': '40.724873', 'longitude': '-73.99...",NaN,588 BROADWAY,NaN,MANHATTAN,10012.0,1,0,0,0,0,0,0,0,E-Scooter,Driver Inattention/Distraction,Sedan,Passenger Distraction
58429,2022-12-30 07:35:00,2022-12-30,7:35,4598426,40.634050,-73.985980,"{'latitude': '40.63405', 'longitude': '-73.985...",15 AVENUE,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,Taxi,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,E-Bike,Unspecified


In [14]:
len(filtered_df)

1874

In [12]:
# Create a boolean mask for rows where injuries do not match
mask = df['number_of_persons_injured'] != df['number_of_motorist_injured']

# Use the mask to filter the DataFrame and get rows where injuries do not match
non_matching_injuries = df[mask]

# To display rows where injuries do not match, you can use:
non_matching_injuries


,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
17,2022-01-01 17:00:00,2022-01-01,17:00,4491448,40.596745,-73.985275,"{'latitude': '40.596745', 'longitude': '-73.98...",86 STREET,NaN,STILLWELL AVENUE,BROOKLYN,11214.0,1,0,0,0,1,0,0,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,E-Bike,Driver Inattention/Distraction
40,2022-01-01 18:30:00,2022-01-01,18:30,4491546,40.564285,-74.126520,"{'latitude': '40.564285', 'longitude': '-74.12...",NaN,110 GUYON AVENUE,NaN,STATEN ISLAND,10306.0,1,0,0,0,1,0,0,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Bike,Unspecified
70,2022-01-01 20:08:00,2022-01-01,20:08,4491446,40.802074,-73.949790,"{'latitude': '40.802074', 'longitude': '-73.94...",NaN,55 WEST 116 STREET,NaN,MANHATTAN,10026.0,1,0,0,0,1,0,0,0,Sedan,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,Bike,View Obstructed/Limited
81,2022-01-01 21:15:00,2022-01-01,21:15,4491350,40.739920,-73.815130,"{'latitude': '40.73992', 'longitude': '-73.815...",KISSENA BOULEVARD,NaN,NaN,NaN,NaN,1,0,0,0,1,0,0,0,Station Wagon/Sport Utility Vehicle,Failure to Yield Right-of-Way,Bike,Passing Too Closely
89,2022-01-01 17:00:00,2022-01-01,17:00,4491407,40.767593,-73.912000,"{'latitude': '40.767593', 'longitude': '-73.91...",NaN,25-02 STEINWAY STREET,NaN,QUEENS,11103.0,1,0,0,0,1,0,0,0,Sedan,Backing Unsafely,Bike,Unspecified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58425,2022-12-30 16:23:00,2022-12-30,16:23,4597190,40.673650,-73.874960,"{'latitude': '40.67365', 'longitude': '-73.874...",FOUNTAIN AVENUE,NaN,BELMONT AVENUE,BROOKLYN,11208.0,1,0,0,0,1,0,0,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Bike,Unspecified
58429,2022-12-30 07:35:00,2022-12-30,7:35,4598426,40.634050,-73.985980,"{'latitude': '40.63405', 'longitude': '-73.985...",15 AVENUE,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,Taxi,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,E-Bike,Unspecified
58514,2022-12-31 19:57:00,2022-12-31,19:57,4596097,40.613040,-73.926210,"{'latitude': '40.61304', 'longitude': '-73.926...",FLATBUSH AVENUE,NaN,AVENUE S,BROOKLYN,11234.0,1,0,0,0,1,0,0,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Bike,Traffic Control Disregarded
58516,2022-12-31 20:45:00,2022-12-31,20:45,4596082,40.679680,-73.978260,"{'latitude': '40.67968', 'longitude': '-73.978...",5 AVENUE,NaN,PARK PLACE,BROOKLYN,11217.0,1,0,0,0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Passing or Lane Usage Improper,E-Scooter,Unspecified


In [15]:
# Example usage:
info_dataframe = exp.create_info_dataframe(df)  # Assuming data_df is your DataFrame
info_dataframe

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,0,0.0,0,0,45821,datetime64[ns]
1,crash_date,0,0.0,0,0,365,datetime64[ns]
2,crash_time,0,0.0,0,0,1440,object
3,collision_id,0,0.0,0,0,54204,int64
4,latitude,5011,9.0,940,0,23108,float64
5,longitude,5011,9.0,940,0,19573,float64
6,location,5011,9.0,0,0,27539,object
7,on_street_name,12605,23.0,0,0,3509,object
8,borough,19753,36.0,0,0,5,object
9,zip_code,19761,36.0,0,0,200,float64
